<a href="https://colab.research.google.com/github/adamserag1/Interpretability-for-VRDU-models/blob/main/finetuning/BROS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#BROS

In [1]:
# UNCOMMNET FOR USE IN COLAB
!git clone https://github.com/adamserag1/Interpretability-for-VRDU-models.git

Cloning into 'Interpretability-for-VRDU-models'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 52 (delta 16), reused 15 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (52/52), 689.53 KiB | 1.41 MiB/s, done.
Resolving deltas: 100% (16/16), done.


In [2]:
!pip install transformers torch datasets evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 40.9 MB/s eta 0:00:00
 

In [3]:
import torch
from PIL import Image,ImageDraw, ImageFont
from datasets import load_dataset
import pandas as pd
import evaluate
from transformers import BrosProcessor, BrosSpadeEEForTokenClassification, AutoTokenizer

## Data

In [5]:
funsd = load_dataset("nielsr/funsd", trust_remote_code=True)
label_list = funsd["train"].features["ner_tags"].feature.names
id2label = {v:k for v,k in enumerate(label_list)}
label2id = {k:v for v,k in enumerate(label_list)}

In [6]:
processor = BrosProcessor.from_pretrained("naver-clova-ocr/bros-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("naver-clova-ocr/bros-base-uncased",do_lower_case=True)

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

In [7]:
sample = funsd["train"][66]
image = Image.open(sample["image_path"]).convert("RGB")
print(sample['words'][])

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-7-089f6ecae3b9>, line 3)

In [ ]:
def normalize_bbox(bbox, width, height):
    return [
        int(1000 * (bbox[0] / width)),
        int(1000 * (bbox[1] / height)),
        int(1000 * (bbox[2] / width)),
        int(1000 * (bbox[3] / height)),
    ]

def tokenize(example):
  words = example["words"]
  bboxes = example["bboxes"]
  ner_tags = example["ner_tags"]

  images = [Image.open(path).convert("RGB") for path in example["image_path"]]
  heights = [image.size[1] for image in images]
  widths = [image.size[0] for image in images]
  normalized_bboxes = []

  for bbox_list, width, height in zip(bboxes, widths, heights):
    normalized_bboxes_per_sample = []
    for bbox in bbox_list:
      normalized_bboxes_per_sample.append(normalize_bbox(bbox, width, height))
    normalized_bboxes.append(normalized_bboxes_per_sample)

  bboxes = normalized_bboxes

  tokenized = tokenizer(
      words,
      is_split_into_words=True,
      padding="max_length",
      max_length = 512,
      truncation=True,
      return_tensors="pt"
  )


  tokenized = {k: v.squeeze(0) for k, v in tokenized.items()}

  labels = []
  word_ids = tokenized.word_ids()
  previous_word_idx = None
  for word_idx in word_ids:
      if word_idx is None:  # Special tokens ([CLS], [SEP], padding)
          labels.append(-100)
      elif word_idx != previous_word_idx:  # First token of a word
          labels.append(ner_tags[word_idx])
      else:  # Subword token
          labels.append(ner_tags[word_idx])  # Same label as first token
      previous_word_idx = word_idx


  # Add labels and normalized bboxes to tokenized inputs
  tokenized["labels"] = torch.tensor(labels)
  tokenized["bboxes"] = torch.tensor(normalized_bboxes, dtype=torch.float)
  return tokenized



In [ ]:
train_dataset = funsd["train"].map(tokenize, batched=True, remove_columns=funsd["train"].column_names)
val_dataset = funsd["test"].map(tokenize, batched=True, remove_columns=funsd["train"].column_names)

In [8]:
def preprocess_data(examples, tokenizer, label_list):
    words = examples["words"]
    bboxes = examples["bboxes"]
    ner_tags = examples["ner_tags"]

    # Tokenize words
    tokenized_inputs = tokenizer(
        words,
        is_split_into_words=True,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt"
    )

    # Convert rectangle bboxes to quadrangle format
    quadrangle_bboxes = []
    for bbox in bboxes:
        quad_bboxes = []
        for box in bbox:
            x0, y0, x1, y1 = box
            quad_box = [x0, y0, x1, y0, x1, y1, x0, y1]  # [top-left, top-right, bottom-right, bottom-left]
            quad_bboxes.append(quad_box)
        quadrangle_bboxes.append(quad_bboxes)

    # Align NER labels and bounding boxes with tokenized inputs
    labels = []
    aligned_bboxes = []

    for i in range(len(words)):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        token_bboxes = []

        for word_idx in word_ids:
            if word_idx is None:  # Special tokens ([CLS], [SEP], [PAD])
                label_ids.append(-100)  # Ignore in loss
                token_bboxes.append([0] * 8)  # Dummy bbox for special tokens
            else:
                # Use ner_tags directly as label IDs
                label_ids.append(ner_tags[i][word_idx])
                token_bboxes.append(quadrangle_bboxes[i][word_idx])

        labels.append(label_ids)
        aligned_bboxes.append(token_bboxes)

    # Convert to tensors
    tokenized_inputs["labels"] = torch.tensor(labels)
    tokenized_inputs["bbox"] = torch.tensor(aligned_bboxes)

    return tokenized_inputs

In [9]:
encoded_funsd_train = funsd["train"].map(
    lambda x: preprocess_data(x, tokenizer, label_list),
    batched=True,
    remove_columns=funsd["train"].column_names
)

encoded_funsd_val = funsd["test"].map(
    lambda x: preprocess_data(x, tokenizer, label_list),
    batched=True,
    remove_columns=funsd["test"].column_names
)


Map:   0%|          | 0/149 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

## MODEL

In [10]:
model = BrosSpadeEEForTokenClassification.from_pretrained("naver-clova-ocr/bros-base-uncased", num_labels=len(label_list), id2label=id2label, label2id=label2id)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BrosSpadeEEForTokenClassification were not initialized from the model checkpoint at naver-clova-ocr/bros-base-uncased and are newly initialized: ['bros.bbox_embeddings.bbox_projection.weight', 'bros.bbox_embeddings.bbox_sinusoid_emb.x_pos_emb.inv_freq', 'bros.bbox_embeddings.bbox_sinusoid_emb.y_pos_emb.inv_freq', 'initial_token_classifier.1.bias', 'initial_token_classifier.1.weight', 'initial_token_classifier.3.bias', 'initial_token_classifier.3.weight', 'subsequent_token_classifier.dummy_node', 'subsequent_token_classifier.key.bias', 'subsequent_token_classifier.key.weight', 'subsequent_token_classifier.query.bias', 'subsequent_token_classifier.query.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BrosSpadeEEForTokenClassification(
  (bros): BrosModel(
    (embeddings): BrosTextEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (bbox_embeddings): BrosBboxEmbeddings(
      (bbox_sinusoid_emb): BrosPositionalEmbedding2D(
        (x_pos_emb): BrosPositionalEmbedding1D()
        (y_pos_emb): BrosPositionalEmbedding1D()
      )
      (bbox_projection): Linear(in_features=192, out_features=64, bias=False)
    )
    (encoder): BrosEncoder(
      (layer): ModuleList(
        (0-11): 12 x BrosLayer(
          (attention): BrosAttention(
            (self): BrosSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (valu

In [11]:
from transformers import TrainingArguments, Trainer, DataCollatorForTokenClassification

training_args = TrainingArguments(
    output_dir="./bros-funsd-finetuned",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

# Data collator
data_collator = DataCollatorForTokenClassification(tokenizer, padding=True)

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [12]:
metric = evaluate.load("seqeval")
def compute_metrics(p):
  predictions, labels = p
  predictions = np.argmax(predictions, axis=2)

  true_preds = [
      [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
      for prediction, label in zip(predictions, labels)
  ]
  true_labels = [
      [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
      for prediction, label in zip(predictions, labels)
  ]
  results = metric.compute(predictions=true_preds, references=true_labels)

  return {
      "precision": results["overall_precision"],
      "recall": results["overall_recall"],
      "f1": results["overall_f1"],
      "accuracy": results["overall_accuracy"],
  }

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_funsd_train,
    eval_dataset=encoded_funsd_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
## ASK CHATGPT WHY NO TRAIN :(
trainer.train()

<ipython-input-14-7f88c6834af3>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


ValueError: The model did not return a loss from the inputs, only the following keys: initial_token_logits,subsequent_token_logits. For reference, the inputs it received are input_ids,token_type_ids,attention_mask,bbox.